### XDA Scraper (with Notes)

In [ ]:
# XDA Scraper without notes

# Scrape S21 posts from XDA-Developers website's S21 page

    # for new models,
        # check whether it was posted within 1 day
        # get post url & comment number
        # go to next page and check again
        # go to each url and get title, contents, # comment


import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import ssl
import time
import datetime


# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE




taglist = []        # title
linklist = []       # url
lastlist = []       # last activity time
datelist = []       # post creation time
labellist = []      # label (Y/N)
comlist = []        # number of comments
modellist = []      # model


###
# For long weekends, simply add the name(s) of day that you missed(ex2), or day up to 2 days ago(ex1) to the timewords below
# ex1. off from Fri to Tue, doing VOC on Wed --> add "Sunday" & "Monday" 
# ex2. off from Wed to Sun, doing VOC on Mon --> add "Wed" & "Thur" 
###
vocday = input("Is it Monday or nah? (Enter y/n):")
if vocday == "y":
    print("Gotcha fam. Take it easy.")
    timewords = ['now', 'second ago', 'A moment ago', '1 minute ago', 'minutes ago', 'hour ago', 'hours ago', 'Today', 'Yesterday', 'Saturday', 'Friday']
else:
    print("Gotcha fam. One day closer to Friday, amirite?")
    timewords = ['now', 'second ago', 'A moment ago', '1 minute ago', 'minutes ago', 'hour ago', 'hours ago', 'Today', 'Yesterday']


time.sleep(2.5)


## # How to get today's date & time
## # https://www.programiz.com/python-programming/datetime/current-datetime
##from datetime import date
##print("Today is ", date.today())
##print("Right now is", datetime.now())
##dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
##print("date and time =", dt_string)
##
## # https://www.hellocodeclub.com/python-get-day-of-week/
##import datetime      
##week_days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
##week_num=datetime.date(2021,8,5).weekday()
##print(week_days[week_num])



# https://stackoverflow.com/questions/16627227/problem-http-error-403-in-python-3-web-scraping
# urllib.error.HTTPError: HTTP Error 403: Forbidden
# Need a different way to get around the urllib web scraper block

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"
opener = AppURLopener()
#response = opener.open('http://httpbin.org/user-agent')

# define a function for retrieving titles & post links on a results page
def scraperesults(searchpage, timewords):
    
    print("Scraping a new model/page...")
    # url = searchpage
    response = opener.open(searchpage)
    #html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(response, "html.parser")

    # Determine the latest activity on the posts
    threads = soup.find_all("div", class_=
            re.compile(r"structItem[ \t]+structItem--thread[ \t]+is-prefix[0-9]+[ \t]+js-inlineModContainer[ \t]+js-threadListItem-[0-9]+"))   # find all threads. Remove ', text=True' ! 
    
    for thread in threads:
        lastacttime = thread.find("time", class_=re.compile(r'structItem-latestDate[ \t]+u-dt')).text
        #print(lastacttime)

        # https://stackoverflow.com/questions/3271478/check-list-of-words-in-another-string
        if any(word in lastacttime for word in timewords):                      # if any of the keywords is found as latest comment time,
            taglist.append(thread.find_all('a', text=True)[1].text)             # scrape title
            #print(thread.find_all('a', text=True)[1].text)     # find 2nd 'a' element under post
            
            linklist.append("https://forum.xda-developers.com" + thread.find_all('a', text=True)[1]['href'])    # scrape hyperlink
            # print(thread.find_all('a', text=True)[1]['href'])
            # print(thread.find_all('a', text=True)[0]['href']) # this also works
            
            lastlist.append(lastacttime)                                        # scrape last activity time
            # print(lastacttime)

            datelist.append(thread.find_all('a', text=True)[3].text)            # scrape post creation time
            #print(thread.find_all('a', text=True)[3].text)

            comlist.append(int(''.join(re.findall('[0-9]+', str(thread.find('dd'))))))       # scrape number of comments
            #print(int(''.join(re.findall('[0-9]+', str(thread.find('dd'))))))       # re.findall(blah) gives ['5']. use ''.join(blah) to unlist the number as string

            # can't tell model by looking at final URL; strip model name from the front-page-of-the-board URL
            #print(searchpage.lstrip("https://forum.xda-developers.com/f/samsung-galaxy-").rstrip(r".[0-9]+/|.[0-9]+/page-[0-9]").replace("-"," "))
            #input()
            modellist.append(searchpage.lstrip("https://forum.xda-developers.com/f/samsung-galaxy-").rstrip(r".[0-9]+/|.[0-9]+/page-[0-9]").replace("-"," "))
            


    print("-----------Going to next page...----------")
    time.sleep(0.5)   # add 0.5s delay



# for new, major S-line models, do 1-3 pages
searchpages = ["https://forum.xda-developers.com/f/samsung-galaxy-s22-ultra.12515/",
               "https://forum.xda-developers.com/f/samsung-galaxy-s22-ultra.12515/page-2",
               "https://forum.xda-developers.com/f/samsung-galaxy-s22-ultra.12515/page-3",
               "https://forum.xda-developers.com/f/samsung-galaxy-tab-s8.12521/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-fold-4.12657/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-fold-4.12657/page-2",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-flip-4.12655/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-flip-4.12655/page-2",
               # "https://forum.xda-developers.com/f/samsung-galaxy-s21-fe.12389/",
               "https://forum.xda-developers.com/f/samsung-galaxy-s21.11937/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-fold3.12349/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-flip-3.12351/"
               ]


# scrape page 1~4 of S21 and page 1~2 of Fold, ZFlip & S20(FE) for post links

for searchpage in searchpages:
    scraperesults(searchpage, timewords)


print("\nNumber of taglist: ", len(taglist), "posts",
      "\nNumber of linklist: ", len(linklist), "posts")

# input()






bodylist = []


# Open up all links and get data (Crawl)

for hyperlink in linklist:
    # url = hyperlink
    response = opener.open(hyperlink)
    # html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(response, "html.parser")

    print("Scraping...")

    try:                # if there's any error such as "ValueError: read of closed file" skip over the error with try/except. Press enter in prompt to continue
        # Body text
        # body = soup.find("article", class_="message-body[ \t]+js-selectToQuote") # .find("br", text=True)
        body = soup.find("div", class_="bbWrapper")
        # if body.find("p") != None:
        #       print(body.find_all("p"))
        bodylist.append(body.text.strip())   # remove whitespace at beginning & end
        # print(body.text.strip())
    except:
        pass


    time.sleep(0.5)  # add 0.5s delay




    # instead of detecting multiple pages on post and navigating to last page + last comment,
    # on results page, detect # of comments and latest comment date
    # follow latest commenter's profile, and look at date of "Re: " + title

    # Last comment date


    # ========= forget the comment count method below, and focus on detecting latest comment =======
##    
##    if comm >= 1 && comm < 10:        # page 2 forms at 10 comments
##        # fetch the comment dates
##        lastdatec = soup.find_all('span', class_='DateTime lia-message-posted-on lia-component-common-widget-date')     # not list. It's bs4.element.ResultSet, each entry bs4.element.Tag
##        lastdate = datetime.fromtimestamp(0)      # create a datetime object to compare last post time to
##
##        i=0
##        
##        for date in lastdatec:      # find the latest date
##            i += 1
##            print(i)
##            if i == len(lastdatec):
##                # for old posts, class_="local-date" & class_="local-time" are shown, with times written under .text
##                if date.find("span", class_="local-date"):
##                    date = date.find("span", class_="local-date").text + " " + date.find("span", class_="local-time").text
##                    date = datetime.strptime(date.strip('\u200e'), '%m-%d-%Y %I:%M %p')
##
##                # for newer posts, class_="local-friendly-date" is shown, with time written inside title="08-01-2021 09:26 AM"   
##                else:      # date.find("span")['title'] exists instead
##                    date = date.find("span")['title']
##                    date = datetime.strptime(date, '\u200e%m-%d-%Y %I:%M %p')   # or (startdate.strip('\u200e'), '%m-%d-%Y %I:%M %p')
##                print(date)     # class 'datetime.datetime'
##                
##                if date > lastdate:
##                    lastdate = date
##        i += 1
##
##        lastlist.append(lastdate)
##    else:
##        lastdate = '-'
##        lastlist.append(lastdate)

##   lolol = input()





# Create DataFrame and export to Excel file

import pandas as pd

# print(len(taglist), len(linklist), len(datelist), len(bodylist), len(comlist))

dictio = {'Model' : modellist,
          'Priority' : ["C"]*len(taglist),  # create a list of C's
          'Title' : taglist,
          'Body Text' : bodylist,
          'Thread' : [1]*len(taglist),   # create a list of 1's
          'Comments' : comlist,
          'URLs' : linklist,
          'Start Date' : datelist,
          'Last Date' : lastlist,
          'Issue?' : ['Y']*len(taglist)   # for now, make them all Y
          }
df = pd.DataFrame(dictio)
print(df)

input()


# Post-processing DataFrame
# For models other than S21, if 0 comment, drop the data


# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
# https://stackoverflow.com/questions/28876243/how-to-delete-the-current-row-in-pandas-dataframe-during-df-iterrows

# if any portion of the hyperlink matches any of those in zerocommentpages AND there's 0 comment, drop that row
#for indexx, roww in df.iterrows():
#    if roww['Comments'] == 0:
#        if any(page in roww['URLs'] for page in zerocommentpages):
#            df.drop(indexx, inplace=True)

"""
for indexx, roww in df.iterrows():
    if roww['Comments'] == 0:
        if "us.community.samsung.com/t5/Galaxy-Fold/" in roww['URLs']:
            df.drop(indexx, inplace=True)
        elif "us.community.samsung.com/t5/Galaxy-Z-Flip/" in roww['URLs']:
            df.drop(indexx, inplace=True)
        elif "us.community.samsung.com/t5/Galaxy-S20/" in roww['URLs']:
            df.drop(indexx, inplace=True)
print(df)
"""


#df2 = pd.read_excel("samsungscrap.xlsx")

#df.append(df2) or concatenate

todaystring = datetime.datetime.now().strftime("%Y%m%d")    # 20220331
filenom = "xdascrap"
fileext = ".xlsx"
filename = str(filenom + todaystring + fileext)

df.to_excel(filename, sheet_name=str(datetime.date.today()))     # Create a new file (can overwrite, so careful)
# df.to_csv('samsungscrap.csv')





# https://stackoverflow.com/questions/47737220/append-dataframe-to-excel-with-pandas

# 2021. 8. 18 Start from here! 
# https://www.guru99.com/python-csv.html
# https://realpython.com/python-csv/


# using import csv, read table from previous file
# find & replace duplicates, update file
    # for N & R, drop the new duplicate
    # for Y, drop the old duplicate



# SQL database update







lol = input()







# for body text,
# body = soup.find_all("p", text=True)
# bodylist.append(body[0].get_text())
# doesn't account for multiple p's (several paragraphs)
# Also, sometimes there is no p at all - just ("div", class_="lia-message-body-content")


### Scrape 5 posts from Samsung Community website
##
##    # scrape a page
##    # scrape from a results page
##    # scrape posts based on date
##    # write to CSV
##    # save to database
##
##
##
### scrape a particular page
##url = "https://us.community.samsung.com/t5/Galaxy-Z-Flip/Whatsapp-s-Picture-in-Picture-Video-Call-Screen-Does-Not-Show/m-p/1843314"
##html = urlopen(url, context=ctx).read()
##soup = BeautifulSoup(html, "html.parser")
##
##
### Retrieve all of the h1 tags
##tags = soup('h1', '_eYtD2XCVieq6emjKBH3m')
##print(len(tags))    # why are there 2, not 1?
##for tag in tags:
##    # Look at the parts of a tag
##    # print('TAG:', tag)
##    # print('URL:', tag.get('href', None))
##    print('Title:', tag.contents[0])
##    # print('Attrs:', tag.attrs)
##
##
##tags1 = soup('p', '_1qeIAgB0cPwnLhDF9XSiJM')    # length 5: body & comments
##tags1 = tags1[0]
##print('Body:', tags1.contents[0])
##
##
##tags2 = soup('span', 'FHCV02u6Cp2zYL0fhQPsO')   # length 1
##print('Number of Comments:', tags2[0].contents[0].strip(" coments"))
### From "3 comments", remove whitespace, c, o, m, e, n, t, s from both ends of string
##
##
##tags3 = soup('a', '_3jOxDPIQ0KaOWpzvSQo-1s')
##print('Hyperlink:', tags3[0].get('href', None))
##
##
##
##
### results page scrape for zflip
### https://us.community.samsung.com/t5/Galaxy-Z-Flip/bd-p/get-help-galaxy-ZFlip
##




### Attempt at figuring out last comment date by going to last page + last comment
##    templist = []
##    if soup.find('ul', class_"lia-paging-full-pages"):   # multiple pages of comments
##        pages = soup.find('ul', class_"lia-paging-full-pages")
##        pagelen = len(pages)
##        nexturl = pages.find('li', class_='lia-paging-page-last lia-js-data-pageNum-' + pagelen)['href']
##        templist.append(
##    
##    elif comm >= 1 && comm < 10:        # page 2 forms at 10 comments
##        # fetch the comment dates
##        lastdatec = soup.find_all('span', class_='DateTime lia-message-posted-on lia-component-common-widget-date')     # not list. It's bs4.element.ResultSet, each entry bs4.element.Tag
##        lastdate = datetime.fromtimestamp(0)      # create a datetime object to compare last post time to
##        print(lastdate)
##        print(datetime.fromtimestamp(1))
##        i=0
##        print(len(lastdatec))
##        for date in lastdatec:      # find the latest date
##
##            # for old posts, class_="local-date" & class_="local-time" are shown, with times written under .text
##            if date.find("span", class_="local-date"):
##                date = date.find("span", class_="local-date").text + " " + date.find("span", class_="local-time").text
##                date = datetime.strptime(date.strip('\u200e'), '%m-%d-%Y %I:%M %p')
##
##            # for newer posts, class_="local-friendly-date" is shown, with time written inside title="08-01-2021 09:26 AM"   
##            else:      # date.find("span")['title'] exists instead
##                date = date.find("span")['title']
##                date = datetime.strptime(date, '\u200e%m-%d-%Y %I:%M %p')   # or (startdate.strip('\u200e'), '%m-%d-%Y %I:%M %p')
##            print(date)     # class 'datetime.datetime'
##            
##            if date > lastdate:
##                lastdate = date
##
##        lastlist.append(lastdate)
##    else:
##        lastdate = '-'
##        lastlist.append(lastdate)



### go to filtered search results page (shows less S21 issues)
##url = "https://us.community.samsung.com/t5/forums/searchpage/tab/message?filter=location&q=s21&noSynonym=false&location=forum-board:GalaxyS21&sort_by=-topicPostDate&collapse_discussion=true"
##html = urlopen(url, context=ctx).read()
##soup = BeautifulSoup(html, "html.parser")
##
### Retrieve all of the post links
##tags = soup.find_all("a", class_="page-link lia-link-navigation lia-custom-event")
##print(tags)
##lol = input()


### (Old method) Retrieve all of the post titles & links
##tags = soup.find_all("h3", text=True)
### print(tags)
##
##for eachtag in tags:
##    # print(eachtag.text)
##    taglist.append(eachtag.text)
##    linklist.append(eachtag.find("a")['href'])





# https://stackoverflow.com/questions/23380171/using-beautifulsoup-to-extract-text-without-tags
##### Is this more efficient? :      -> Yes!        2021.8.4.
##threads = soup.find_all("article", class_="samsung-message-tile samsung-thread-unread")   # find all threads. Remove ', text=True' ! 
##print(len(threads))
##
##for thread in threads:
##    lastacttime = thread.find("time", datetime='true').text
##    timewords = ['now', 'second', 'm ago', 'hour', 'yesterday']
##
##    # https://stackoverflow.com/questions/3271478/check-list-of-words-in-another-string
##    if any(word in lastacttime for word in timewords):                      # if any of the keywords is found as latest comment time,
##        taglist.append(thread.find('h3', text=True).text)                   # scrape title
##        print(thread.find('h3', text=True).text)
##        
##        linklist.append(thread.find('h3', text=True).find('a')['href'])     # scrape hyperlink
##        print(thread.find('h3', text=True).find('a')['href'])
##        
##        lastlist.append(lastacttime)                                        # scrape last activity time
##        print(lastacttime)
#####


# Problem when trying to append sheets to existing .xlsx file
# zipfile.BadZipFile: File is not a zip file error  ->  https://python-forum.io/thread-32833.html
# check pandas version with: pip show pandas
# reinstall pandas version with: pip install pandas==1.1.5
# https://stackoverflow.com/questions/31013308/bad-zip-file-error-when-rewriting-code-in-python-3-4
# https://stackoverflow.com/questions/28058563/write-to-stringio-object-using-pandas-excelwriter
# https://github.com/pandas-dev/pandas/issues/28653

#try:
#    with pd.ExcelWriter("samsungscrap.xlsx", mode="a", engine="xlsxwriter") as writer:
#        df.to_excel(writer, sheet_name=str(date.today()))
#except FileNotFoundError:
#    df.to_excel("samsungscrap.xlsx", sheet_name=str(date.today()))



### XDA Scraper (without Notes)

In [ ]:
# XDA scraper

# Scrape posts about Samsung's mobile devices from XDA-Developers website
    # for new models,
        # check whether it was posted within 1 day
        # get post url & comment number
        # go to next page and check again
        # go to each url and get title, contents, # comment


import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import ssl
import time
import datetime
import pandas as pd


# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


taglist = []        # title
linklist = []       # url
lastlist = []       # last activity time
datelist = []       # post creation time
labellist = []      # label (Y/N)
comlist = []        # number of comments
modellist = []      # device model


###
# For long weekends, simply add the name(s) of day that you missed(ex2), or day up to 2 days ago(ex1) to the timewords below
# ex1. off from Fri to Tue, doing VOC on Wed --> add "Sunday" & "Monday" 
# ex2. off from Wed to Sun, doing VOC on Mon --> add "Wed" & "Thur" 
###
vocday = input("Is it Monday or nah? (Enter y/n):")
if vocday == "y":
    print("Gotcha fam. Take it easy.")
    timewords = ['now', 'second ago', 'A moment ago', '1 minute ago', 'minutes ago', 'hour ago', 'hours ago', 'Today', 'Yesterday', 'Saturday', 'Friday']
else:
    print("Gotcha fam. One day closer to Friday, amirite?")
    timewords = ['now', 'second ago', 'A moment ago', '1 minute ago', 'minutes ago', 'hour ago', 'hours ago', 'Today', 'Yesterday']


time.sleep(2.5)


class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"
opener = AppURLopener()
#response = opener.open('http://httpbin.org/user-agent')


# define a function for retrieving titles & post links on a results page
def scraperesults(searchpage, timewords):
    
    print("Scraping a new model/page...")
    response = opener.open(searchpage)
    soup = BeautifulSoup(response, "html.parser")

    # Determine the latest activity on the posts
    threads = soup.find_all("div", class_=
            re.compile(r"structItem[ \t]+structItem--thread[ \t]+is-prefix[0-9]+[ \t]+js-inlineModContainer[ \t]+js-threadListItem-[0-9]+"))   # find all threads. Remove ', text=True' ! 
    
    for thread in threads:
        lastacttime = thread.find("time", class_=re.compile(r'structItem-latestDate[ \t]+u-dt')).text
        #print(lastacttime)

        # https://stackoverflow.com/questions/3271478/check-list-of-words-in-another-string
        if any(word in lastacttime for word in timewords):      # if any of the keywords is found as latest comment time,
            taglist.append(thread.find_all('a', text=True)[1].text)             # scrape title
            #print(thread.find_all('a', text=True)[1].text)     # find 2nd 'a' element under post
            
            linklist.append("https://forum.xda-developers.com" + thread.find_all('a', text=True)[1]['href'])    # scrape hyperlink            
            lastlist.append(lastacttime)                                        # scrape last activity time
            datelist.append(thread.find_all('a', text=True)[3].text)            # scrape post creation time
            comlist.append(int(''.join(re.findall('[0-9]+', str(thread.find('dd'))))))       # scrape number of comments
            
            # can't tell model by looking at final URL; strip model name from the front-page-of-the-board URL
            modellist.append(searchpage.lstrip("https://forum.xda-developers.com/f/samsung-galaxy-").rstrip(r".[0-9]+/|.[0-9]+/page-[0-9]").replace("-"," "))
            


    print("-----------Going to next page...----------")
    time.sleep(0.5)   # add 0.5s delay



# for new, major S-line models, do 1-3 pages
searchpages = ["https://forum.xda-developers.com/f/samsung-galaxy-s22-ultra.12515/",
               "https://forum.xda-developers.com/f/samsung-galaxy-s22-ultra.12515/page-2",
               "https://forum.xda-developers.com/f/samsung-galaxy-s22-ultra.12515/page-3",
               "https://forum.xda-developers.com/f/samsung-galaxy-tab-s8.12521/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-fold-4.12657/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-fold-4.12657/page-2",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-flip-4.12655/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-flip-4.12655/page-2",
               # "https://forum.xda-developers.com/f/samsung-galaxy-s21-fe.12389/",
               "https://forum.xda-developers.com/f/samsung-galaxy-s21.11937/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-fold3.12349/",
               "https://forum.xda-developers.com/f/samsung-galaxy-z-flip-3.12351/"
               ]


for searchpage in searchpages:
    scraperesults(searchpage, timewords)


print("\nNumber of taglist: ", len(taglist), "posts",
      "\nNumber of linklist: ", len(linklist), "posts")


bodylist = []


# Open up all links and get data (Crawl)

for hyperlink in linklist:
    response = opener.open(hyperlink)
    soup = BeautifulSoup(response, "html.parser")

    print("Scraping...")

    try:       # for errors like "ValueError: read of closed file" skip with try/except. Press enter to continue
        body = soup.find("div", class_="bbWrapper")
        bodylist.append(body.text.strip())   # remove whitespace at beginning & end
    except:
        pass

    time.sleep(0.5)      # add 0.5s delay



# Create DataFrame and export to Excel file

dictio = {'Model' : modellist,
          'Priority' : ["C"]*len(taglist),  # create a list of C's
          'Title' : taglist,
          'Body Text' : bodylist,
          'Thread' : [1]*len(taglist),   # create a list of 1's
          'Comments' : comlist,
          'URLs' : linklist,
          'Start Date' : datelist,
          'Last Date' : lastlist,
          'Issue?' : ['Y']*len(taglist)   # as placeholder, make them all Y
          }
df = pd.DataFrame(dictio)
print(df)

input()


todaystring = datetime.datetime.now().strftime("%Y%m%d")    # 20220331
filenom = "xdascrap"
fileext = ".xlsx"
filename = str(filenom + todaystring + fileext)

df.to_excel(filename, sheet_name=str(datetime.date.today()))     # Create a new file (can overwrite, so careful)

lol = input()



# Ideas for improvement & new features
    # Post-processing DataFrame
    # using import csv, read table from previous file
        # find & replace duplicates, update file
        # for N & R, drop the new duplicate
        # for Y, drop the old duplicate
    # SQL database update




### L

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5caba9f-cd36-4d50-aaa3-2cf59957a2f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>